# pdfの英語文献から，出現頻度順の日訳英単語リストを作成するプログラム
## INPUT : PDF 英語文献     
## OUTPUT : CSV 翻訳 word list

### 品詞の略については以下サイトを参照
[Qiita](https://qiita.com/m__k/items/ffd3b7774f2fde1083fa "NLTKの使い方をいろいろ調べてみた")

### 初期設定
#### 最初の一回のみ，実行する．その後はコメントアウトしてよい．

In [ ]:
# # 使用するライブラリのインストール
# !pip install PyPDF2
# !pip install nltk
# !pip install deep_translator
# !pip install tqdm

# # nltkライブラリの事前ダウンロードファイル
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [ ]:
import PyPDF2

# INPUT: pdf_file に 読み込むパスを記述！！！

In [ ]:
# pdf 読み込み，textに変換．
pdf_file = "../RNA Viral Community in Human Feces.pdf"
with open(pdf_file, "rb") as f:
    reader = PyPDF2.PdfFileReader(f)
    num_page = reader.getNumPages()
    pages = []
    for i in range(num_page):
        pages.append(reader.getPage(i).extractText())


### 単語の整理: 単語群をもっと整えたい場合は，以下を編集するとよいかも
### ほしい品詞なども設定できる

In [ ]:
import nltk
import numpy as np
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

word_lists = []
for i in range(num_page):
    word_list = nltk.word_tokenize(pages[i])
    word_lists.extend(word_list)

# いらない単語を削除
removed_word_list = []
for word in word_lists:
    if word.isalpha():
        removed_word_list.append(word)
        
        
word_dict = {} # 重複回数をカウント
for word in removed_word_list:
    if not word in word_dict:
        word_dict[word] = 1
    else:
        word_dict[word] += 1
                

sorted_dict = sorted(word_dict.items(), key=lambda x:x[1], reverse=True)

# 品詞の取得
pos = nltk.pos_tag([pair[0] for pair in sorted_dict])

print(len(pos))
# 欲しい品詞のポジションを取得
p = []
for i, pair in enumerate(pos):
    po = pair[1]
    if any(x in po for x in ["FW", "JJ", "LS", "MD", "NN", "RB", "RP", "VB"]):
        p.append(i)

sorted_dict = np.array(sorted_dict)[p]
pos = np.array(pos)[p]

print(len(p))
print(sorted_dict.shape)
print(pos.shape)
print((pos))


### 英日訳 1秒間隔で英日訳を取得
### 単語数が少ない場合は，sleepを消してもよい．
### 単語数が多く，sleepが短いと，接続がキャンセルされる．

In [ ]:
from deep_translator import GoogleTranslator
from tqdm.notebook import tqdm
from time import sleep
translator = GoogleTranslator(sourse='en', target='ja')

engs = []
for i, word in tqdm(enumerate(sorted_dict)):
    translated = translator.translate(word[0])
    engs.append(translated)
    print("{}\t{}\t{}\t{}\t{}".format(sorted_dict[i][1], sorted_dict[i][0], pos[i][0], pos[i][1], translated))
    sleep(1)
    

## OUTPUT: save_fileに保存先パスを記述！！！

In [ ]:
save_file = os.path.splitext(pdf_file)[0] + "_word_list" + ".csv"

with open(save_file, "w", encoding="utf-8_sig") as f:
    f.write("出現回数," + "英語," + "品詞," + "日訳" + "\n")
    for i, tup in tqdm(enumerate(sorted_dict)):
        f.write("{},{},{},{}\n".format(tup[1], tup[0], pos[i][1], engs[i]))
        